In [ ]:
import tifffile
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from skimage import exposure

# Paths
input_dir = Path("raw-data/train")

# Grab first .tif
tif_path = next(input_dir.rglob("*.tif"))
print(f"Using {tif_path}")

# Enhancement functions
def enhance_contrast(img, low_pct=1, high_pct=99):
    p_low, p_high = np.percentile(img, (low_pct, high_pct))
    return exposure.rescale_intensity(img, in_range=(p_low, p_high))

def clahe_enhance(img, clip=0.03):
    m = np.max(img)
    img_norm = img / m if m > 0 else img
    return exposure.equalize_adapthist(img_norm, clip_limit=clip)

# Load TIFF
with tifffile.TiffFile(tif_path) as tif:
    arr = tif.asarray()

bright_field_index = 0
fluorescence_index = 1

if arr.ndim == 4:
    bf_image = arr[0, bright_field_index]
    fl_image = arr[0, fluorescence_index]
elif arr.ndim == 3 and arr.shape[0] == 2:
    bf_image = arr[bright_field_index]
    fl_image = arr[fluorescence_index]
else:
    raise ValueError(f"Unsupported shape: {arr.shape}")

# Processed versions
bf_stretch_mild = enhance_contrast(bf_image, 5, 95)
bf_stretch_med  = enhance_contrast(bf_image, 2, 98)
bf_stretch_aggr = enhance_contrast(bf_image, 1, 99)
bf_clahe        = clahe_enhance(bf_image)

fl_stretch_mild = enhance_contrast(fl_image, 5, 95)
fl_stretch_med  = enhance_contrast(fl_image, 2, 98)
fl_stretch_aggr = enhance_contrast(fl_image, 1, 99)
fl_clahe    = clahe_enhance(fl_image)

# Show results
rows = [
    ("BF Raw",          bf_image,       "FL Raw",          fl_image),
    ("BF 5–95% stretch",bf_stretch_mild,"FL 5–95% stretch",fl_stretch_mild),
    ("BF 2–98% stretch",bf_stretch_med, "FL 2–98% stretch",fl_stretch_med),
    ("BF 1–99% stretch",bf_stretch_aggr,"FL 1–99% stretch",fl_stretch_aggr),
    ("BF CLAHE",        bf_clahe,       "FL CLAHE",        fl_clahe),
]

fig, axes = plt.subplots(len(rows), 2, figsize=(10, 4*len(rows)))
for r, (title_bf, img_bf, title_fl, img_fl) in enumerate(rows):
    axes[r, 0].imshow(img_bf, cmap="gray")
    axes[r, 0].set_title(title_bf)
    axes[r, 0].axis("off")

    axes[r, 1].imshow(img_fl, cmap="magma")
    axes[r, 1].set_title(title_fl)
    axes[r, 1].axis("off")

plt.tight_layout()
plt.show()